## Teste de Ciências de Dados - Cadastra
### Etapa 2 - Modelagem e Avaliação 

### Autor: Leonardo Padilha de Lima
#### (15) 99738-3447
#### leo.plima08@gmail.com

 https://github.com/leoplima/desafio_cadastra


### 1)	Importação das bibliotecas necessárias

In [1]:
# importação de pacotes básico

import pandas as pd

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import xgboost as xgb
from sklearn.feature_selection import SelectFromModel
import joblib
import matplotlib.pyplot as plt

import warnings


%matplotlib inline

warnings.filterwarnings("ignore")

### 2)	Importação das bases de dados fornecidas usando o Pandas e mostrando as cinco primeiras linhas do conjunto de dados.

In [2]:
# importando base de dados

def importar_base():
    '''
    Função que vai importar a base e transformar em uma DataFrame pandas.
    '''
    try:
    
        df = pd.read_csv('base_eda.csv', sep = ',')
        df = df.sort_values('COD_CICLO').reset_index(drop=True)
    
    except Exception as erro:
        print(f'Opa! Um problema de {erro.__class__} foi encontrato.')
    
    return df

In [3]:
df = importar_base()
df.head()

,COD_CICLO,FLG_DATA,COD_MATERIAL,COD_CANAL,DES_CATEGORIA_MATERIAL,DES_MARCA_MATERIAL,COD_REGIAO,QT_VENDA_BRUTO,QT_DEVOLUCAO,VL_RECEITA_BRUTA,VL_RECEITA_LIQUIDA,FLG_CAMPANHA_MKT_A,FLG_CAMPANHA_MKT_B,FLG_CAMPANHA_MKT_C,FLG_CAMPANHA_MKT_D,FLG_CAMPANHA_MKT_E,PCT_DESCONTO,VL_PRECO
0,201801,0,443142,anon_S0,anon_S5,anon_S20,anon_S10,10068.0,294.0,294270.60,281610.18,0,1,0,0,0,10.0,413.4
1,201801,0,147546,anon_S7,anon_S5,anon_S34,anon_S1,240.0,NaN,413.40,413.40,0,0,0,0,0,NaN,413.4
2,201801,0,420630,anon_S7,anon_S5,anon_S34,anon_S10,2388.0,NaN,43003.08,43003.08,0,1,0,0,0,10.0,353.4
3,201801,0,170454,anon_S7,anon_S5,anon_S24,anon_S10,3516.0,NaN,61451.04,61451.04,0,0,0,0,0,NaN,347.4
4,201801,0,152700,anon_S0,anon_S12,anon_S34,anon_S10,29670.0,1266.0,2352170.40,2210035.02,0,1,0,0,0,10.0,713.4


### 3)	Préprocessamento

#### 3.1) Definir as variáveis preditoras (X) e alvo (y)

In [4]:
X = df.drop('QT_VENDA_BRUTO', axis = 1)
y = df['QT_VENDA_BRUTO']

#### 3.2) Separar em conjunto de treino e teste

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 78)

#### 3.3) Identificar as variáveis para tratamento específicos

In [6]:
# Vamos fazer tratamentos específicos nas variáveis categóricas e numéricas

onehot_encoded = ['COD_CANAL', 'DES_CATEGORIA_MATERIAL', 'DES_MARCA_MATERIAL', 'COD_REGIAO']
imputer = ['QT_DEVOLUCAO','PCT_DESCONTO']
sc = ['QT_DEVOLUCAO','VL_RECEITA_BRUTA','VL_RECEITA_LIQUIDA','PCT_DESCONTO','VL_PRECO']

#### 3.4) Criando os pipeline de pré-processamento

In [7]:
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown = 'ignore'))  # Codifica variáveis categóricas
])

int_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value = 0)) # Imputa valores com zero
])

scaler = Pipeline(steps=[
    ('scaler', StandardScaler()) # realiza o escalonamento
])

#### 3.5) Criando o transformer para os pipelines

In [9]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, onehot_encoded),
        ('imp', int_transformer, imputer),
        ('sca', scaler, sc)
    ])

### 4) Executando pipeline e treinando modelo (base)

In [10]:
# Treinar o modelo base
pipeline_base = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgb.XGBRegressor(random_state = 78))
])

pipeline_base.fit(X_train, y_train)

# AVALIANDO MODELO DE TREINO
y_pred_treino = pipeline_base.predict(X_train)

rmse_treino = mean_squared_error(y_train, y_pred_treino, squared = False)  # RMSE
mape_treino = mean_absolute_percentage_error(y_train, y_pred_treino)  # MAPE
mae_treino = mean_absolute_error(y_train, y_pred_treino) #MAE

print(f"Modelo Base - RMSE: {rmse_treino}, MAPE: {mape_treino}, MAE:{mae_treino}")
print(f"Mediana do Preço: {df['QT_VENDA_BRUTO'].median()}")


Modelo Base - RMSE: 4439.595840295119, MAPE: 0.0843705761617912, MAE:584.8140753987863
Mediana do Preço: 2070.0


### 5) Avaliação do modelo base usando métricas RMSE e MAPE, MAE

In [11]:
# Previsões do modelo base
y_pred_base = pipeline_base.predict(X_test)

rmse_base = mean_squared_error(y_test, y_pred_base, squared=False)  # RMSE
mape_base = mean_absolute_percentage_error(y_test, y_pred_base)  # MAPE
mae_base = mean_absolute_error(y_test, y_pred_base) #MAE

print(f"Modelo Base - RMSE: {rmse_base}, MAPE: {mape_base}, MAE:{mae_base}")
print(f"Mediana do Preço: {df['QT_VENDA_BRUTO'].median()}")

Modelo Base - RMSE: 9903.377124473238, MAPE: 0.0858804975079913, MAE:885.0173356155261
Mediana do Preço: 2070.0


### 6) Otimizando o modelo com seleção de variáveis baseada na importância

In [12]:
# Treinar o modelo base
pipeline_base_selection = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selection', SelectFromModel(xgb.XGBRegressor(random_state=78))),
    ('model', xgb.XGBRegressor(random_state=78))
])

pipeline_base_selection.fit(X_train, y_train)

# AVALIANDO MODELO DE TREINO
y_pred_treino = pipeline_base.predict(X_train)

rmse_treino = mean_squared_error(y_train, y_pred_treino, squared=False)  # RMSE
mape_treino = mean_absolute_percentage_error(y_train, y_pred_treino)  # MAPE
mae_treino = mean_absolute_error(y_train, y_pred_treino) #MAE

print(f"Modelo Base - RMSE: {rmse_treino}, MAPE: {mape_treino}, MAE:{mae_treino}")
print(f"Mediana do Preço: {df['QT_VENDA_BRUTO'].median()}")

Modelo Base - RMSE: 4439.595840295119, MAPE: 0.0843705761617912, MAE:584.8140753987863
Mediana do Preço: 2070.0


### 7) Avaliação do modelo com seleção de variáveis usando métricas RMSE e MAPE, MAE

In [13]:
# Previsões do modelo base
y_pred_base_selection = pipeline_base_selection.predict(X_test)

rmse_base_selection = mean_squared_error(y_test, y_pred_base_selection, squared=False)  # RMSE
mape_base_selection = mean_absolute_percentage_error(y_test, y_pred_base_selection)  # MAPE
mae_base_selection = mean_absolute_error(y_test, y_pred_base_selection) #MAE

print(f"Modelo Base - RMSE: {rmse_base_selection}, MAPE: {mape_base_selection}, MAE:{mae_base_selection}")
print(f"Mediana do Preço: {df['QT_VENDA_BRUTO'].median()}")

Modelo Base - RMSE: 10178.368676785469, MAPE: 0.08891021792897817, MAE:874.7178595182976
Mediana do Preço: 2070.0


### 8) Otimizando o modelo base com RandomizedSearchCV

In [14]:
# Treinar o modelo otimizado com RandomizedSearchCV
pipeline_optimized = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgb.XGBRegressor(random_state=78))
])

# Espaço de busca dos hiperparâmetros
param_dist = {
    'model__objective':['reg:squarederror'],
    'model__booster':['gbtree','gblinear'],
    'model__n_estimators': [100, 300, 500],
    'model__learning_rate': [0.1, 0.3, 0.9, 1],
    'model__max_depth': [3, 5, 7, 9, 10],
    'model__min_child_weight': [1, 2, 3, 4, 5],
    'model__subsample': [0.6, 0.7, 0.9, 1.0],
    'model__colsample_bytree': [0.6, 0.9, 1.0],
    'model__gamma': [0, 0.1, 0.3, 0.4],
    'model__reg_alpha': [0.01, 0.1, 1, 10],
    'model__reg_lambda': [0.1, 1, 10]
}

# Configuração do RandomizedSearchCV
random_search = RandomizedSearchCV(
    pipeline_optimized,
    param_distributions=param_dist,
    n_iter = 5,
    cv = 5,
    random_state = 78,
    n_jobs = -1
)
random_search.fit(X_train, y_train)

# AVALIANDO MODELO DE TREINO
y_pred_treino_ran = random_search.predict(X_train)

rmse_treino = mean_squared_error(y_train, y_pred_treino_ran, squared=False)  # RMSE
mape_treino = mean_absolute_percentage_error(y_train, y_pred_treino_ran)  # MAPE
mae_treino = mean_absolute_error(y_train, y_pred_treino_ran) #MAE

print(f"Modelo Base - RMSE: {rmse_treino}, MAPE: {mape_treino}, MAE:{mae_treino}")
print(f"Mediana do Preço: {df['QT_VENDA_BRUTO'].median()}")



Modelo Base - RMSE: 8398.26098371438, MAPE: 0.3069292894804855, MAE:1146.3811031283417
Mediana do Preço: 2070.0


### 9) Avaliação do modelo com seleção de variáveis usando métricas RMSE e MAPE, MAE

In [15]:
# Avaliação do modelo otimizado usando RMSE e MAPE

y_pred_optimized = random_search.predict(X_test)

rmse_optimized = mean_squared_error(y_test, y_pred_optimized, squared=False)  # RMSE
mape_optimized = mean_absolute_percentage_error(y_test, y_pred_optimized)  # MAPE
mae_optimized = mean_absolute_error(y_test, y_pred_optimized) #MAE

# 8. Comparação entre os modelos

print(f"Modelo Otimizado - RMSE: {rmse_optimized}, MAPE: {mape_optimized}, MAE: {mae_optimized}")

Modelo Otimizado - RMSE: 12768.721251136592, MAPE: 0.30097242154688214, MAE: 1294.4930179523014


### 10) Otimizando o modelo com seleção de variáveis e RandomizedSearchCV

In [16]:

# 7. Treinar o modelo otimizado com RandomizedSearchCV
pipeline_optimized2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selection', SelectFromModel(xgb.XGBRegressor(random_state=78))),
    ('model', xgb.XGBRegressor(random_state=78))
])

# Espaço de busca dos hiperparâmetros
param_dist = {
    'model__objective':['reg:squarederror'],
    'model__booster':['gbtree','gblinear'],
    'model__n_estimators': [100, 300, 500],
    'model__learning_rate': [0.1, 0.3, 0.9, 1],
    'model__max_depth': [3, 5, 7, 9, 10],
    'model__min_child_weight': [1, 2, 3, 4, 5],
    'model__subsample': [0.6, 0.7, 0.9, 1.0],
    'model__colsample_bytree': [0.6, 0.9, 1.0],
    'model__gamma': [0, 0.1, 0.3, 0.4],
    'model__reg_alpha': [0.01, 0.1, 1, 10],
    'model__reg_lambda': [0.1, 1, 10]
}

# Configuração do RandomizedSearchCV
random_search2 = RandomizedSearchCV(
    pipeline_optimized2,
    param_distributions=param_dist,
    n_iter = 5,
    cv = 5,
    random_state = 78,
    n_jobs = -1
)
random_search2.fit(X_train, y_train)


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('cat',
                                                                               Pipeline(steps=[('onehot',
                                                                                                OneHotEncoder(handle_unknown='ignore'))]),
                                                                               ['COD_CANAL',
                                                                                'DES_CATEGORIA_MATERIAL',
                                                                                'DES_MARCA_MATERIAL',
                                                                                'COD_REGIAO']),
                                                                              ('imp',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(fill_value=0,
                                                                                                              strategy='constant'))]),
                                                                               ['QT_DEVOLUCAO',
                                                                                'P...
                                        'model__colsample_bytree': [0.6, 0.9,
                                                                    1.0],
                                        'model__gamma': [0, 0.1, 0.3, 0.4],
                                        'model__learning_rate': [0.1, 0.3, 0.9,
                                                                 1],
                                        'model__max_depth': [3, 5, 7, 9, 10],
                                        'model__min_child_weight': [1, 2, 3, 4,
                                                                    5],
                                        'model__n_estimators': [100, 300, 500],
                                        'model__objective': ['reg:squarederror'],
                                        'model__reg_alpha': [0.01, 0.1, 1, 10],
                                        'model__reg_lambda': [0.1, 1, 10],
                                        'model__subsample': [0.6, 0.7, 0.9,
                                                             1.0]},
                   random_state=78)

### 11) Avaliação final e comparação entre os modelos

In [17]:
# 8. Comparação entre os modelos

y_pred_optimized2 = random_search2.predict(X_test)

# Avaliação do modelo otimizado usando RMSE e MAPE
rmse_optimized2 = mean_squared_error(y_test, y_pred_optimized2, squared=False)  # RMSE
mape_optimized2 = mean_absolute_percentage_error(y_test, y_pred_optimized2)  # MAPE
mae_optimized2 = mean_absolute_error(y_test, y_pred_optimized2) #MAE

print("Comparação entre os Modelos:")
print(f"Modelo Base - RMSE: {rmse_base}, MAPE: {mape_base},  MAE:{mae_base}")
print(f"Modelo Base Selecion - RMSE: {rmse_base_selection}, MAPE: {mape_base_selection}, MAE:{mae_base_selection}")
print(f"Modelo Otimizado - RMSE: {rmse_optimized}, MAPE: {mape_optimized}, MAE: {mae_optimized}")
print(f"Modelo Otimizado Selection - RMSE: {rmse_optimized2}, MAPE: {mape_optimized2}, MAE: {mae_optimized2}")

Comparação entre os Modelos:
Modelo Base - RMSE: 9903.377124473238, MAPE: 0.0858804975079913,  MAE:885.0173356155261
Modelo Base Selecion - RMSE: 10178.368676785469, MAPE: 0.08891021792897817, MAE:874.7178595182976
Modelo Otimizado - RMSE: 12768.721251136592, MAPE: 0.30097242154688214, MAE: 1294.4930179523014
Modelo Otimizado Selection - RMSE: 12616.021605711383, MAPE: 0.3374902052322611, MAE: 1328.0596750266404


### O Ganhador é o modelo base.